In [2]:
import pandas as pd

data = pd.DataFrame(columns = ['Country', 'Year', 'Value'])
df = pd.read_csv('./assets/CO2_Emissions_1960-2018.csv')

# Pays gardés
kept_countries = ['France', 'Italy', 'Spain', 'Germany', 'Portugal', 'United Kingdom']
limit_to_kept_countries = True

df.head(5)

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,204.631696,208.837879,226.081890,214.785217,207.626699,185.213644,172.158729,210.819017,194.917536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,0.906060,0.922474,0.930816,0.940570,0.996033,1.047280,1.033908,1.052204,1.079727,...,1.021954,1.048876,1.005338,1.021646,1.031833,1.041145,0.987393,0.971016,0.959978,0.933541
2,Afghanistan,0.046057,0.053589,0.073721,0.074161,0.086174,0.101285,0.107399,0.123409,0.115142,...,0.211306,0.297065,0.407074,0.335351,0.263716,0.234037,0.232176,0.208857,0.203328,0.200151
3,Africa Western and Central,0.090880,0.095283,0.096612,0.112376,0.133258,0.184803,0.193676,0.189305,0.143989,...,0.426770,0.472819,0.497023,0.490867,0.504655,0.507671,0.480743,0.472959,0.476438,0.515544
4,Angola,0.100835,0.082204,0.210533,0.202739,0.213562,0.205891,0.268937,0.172096,0.289702,...,1.205902,1.221515,1.216317,1.204799,1.261542,1.285365,1.260921,1.227703,1.034317,0.887380


In [3]:
# Récupération de la liste des pays
countries = df['Country Name'].unique().tolist()
print(countries)
df.set_index('Country Name', inplace=True)

['Aruba', 'Africa Eastern and Southern', 'Afghanistan', 'Africa Western and Central', 'Angola', 'Albania', 'Andorra', 'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Central Europe and the Baltics', 'Switzerland', 'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'East Asia & Pacific (excluding high income)', 'Early-demographic dividend', 'East Asia & Pacific'

In [4]:
# Récupération de la liste des années
columns = df.columns
years = columns[1:].tolist()
print(years)

['1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']


In [5]:
# Création nouveau dataframe organisé par pays puis années
data_list = []

for country in countries:
    #values = df[df['Country Name'] == country]
    for year in years:
        value = df.loc[country, year]
        #print(value)
        data_list.append({'Country': country, 'Year': year, 'Value': value})
data = pd.DataFrame(data_list, columns=["Country", "Year", "Value"])
data.head(5)

,Country,Year,Value
0,Aruba,1961,208.837879
1,Aruba,1962,226.081890
2,Aruba,1963,214.785217
3,Aruba,1964,207.626699
4,Aruba,1965,185.213644


In [6]:
data.shape

(15428, 3)

In [7]:
# Suppression des valeurs inutiles si nécessaires
nb_na = data['Value'].isna().sum()
print("Nombre de valeurs invalides ou nulls: " + str(nb_na))
if nb_na > 0:
    data.dropna(subset=['Value'], inplace=True)
data.shape

Nombre de valeurs invalides ou nulls: 2289


(13139, 3)

In [8]:
data.to_csv('./output/CO2_Emissions_sortedByYearAndCountry.csv', index=False)

In [9]:
# Retirer les valeurs des pays différents de cette liste et exporter le fichier
if limit_to_kept_countries:
    data = data[data.Country.isin(kept_countries)]
data.to_csv('./output/CO2_Emissions_sortedByYearAndCountry.csv', index=False)


In [10]:
# Charger le fichier World Energy Consumption
wec = pd.read_csv('./assets/World Energy Consumption.csv')
if limit_to_kept_countries:
    wec = wec[wec.country.isin(kept_countries)]
wec.to_csv('./output/World Energy Consumption (limited).csv', index=False)

In [11]:
# Ajouter une colonne pour insérer les valeurs des émissions de Co2 en métrique tonne
wec['pollution_co2'] = [float("NaN")] * wec.shape[0]
for (i, r) in data.iterrows():
    for (idx, row) in wec.iterrows():
        if row.country == r.Country and int(row.year) == int(r.Year):
            wec.at[idx, 'pollution_co2'] = r.Value
            
# Supprimer la colonne 'population'
wec = wec.drop('population', axis=1, errors='ignore')
wec = wec.drop('iso_code', axis=1, errors='ignore')
wec = wec.drop('gdp', axis=1, errors='ignore')

wec = wec[wec['year'] >= 1960]

# Valeurs enregistrés
wec.to_csv('./output/World Energy Consumption (limited).csv', index=False)

In [204]:
import pandas as pd
from matplotlib import pyplot as plt
# Charger le dataset des taxes sur les énergies
tax = pd.read_csv('./assets/Tax_Energy.csv')

In [205]:
tax.head(5)

,ObjectId,Country,ISO2,ISO3,Indicator,Source,CTS Code,CTS Name,CTS Full Descriptor,Unit,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,4.399314e+10,4.781379e+10,4.754858e+10,5.114559e+10,5.341565e+10,NaN,NaN
1,2,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Percent of GDP,...,NaN,NaN,NaN,3.067206e+00,3.247163e+00,3.066373e+00,3.124865e+00,3.157133e+00,NaN,NaN
2,3,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,3.774111e+10,4.094562e+10,4.040004e+10,4.352182e+10,4.516530e+10,NaN,NaN
3,4,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Percent of GDP,...,NaN,NaN,NaN,2.631314e+00,2.780726e+00,2.605369e+00,2.659072e+00,2.669496e+00,NaN,NaN
4,5,Albania,AL,ALB,Taxes on Pollution,Organisation for Economic Co-operation and Dev...,ECGTEP,Taxes on Pollution,"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,1.782069e+09,1.879970e+09,1.941324e+09,2.226251e+09,2.625011e+09,NaN,NaN


In [206]:
# Regrouper les années dans une seule colonne
tax = pd.melt(tax, id_vars=["ObjectId", "Country", "ISO2", "ISO3", "Indicator", "Source", "CTS Code", "CTS Name", "CTS Full Descriptor", "Unit"], 
                    var_name="Année", value_name="Taxe")


In [207]:
tax.head(5)

,ObjectId,Country,ISO2,ISO3,Indicator,Source,CTS Code,CTS Name,CTS Full Descriptor,Unit,Année,Taxe
0,1,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN
1,2,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Percent of GDP,1995,NaN
2,3,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN
3,4,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Percent of GDP,1995,NaN
4,5,Albania,AL,ALB,Taxes on Pollution,Organisation for Economic Co-operation and Dev...,ECGTEP,Taxes on Pollution,"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN


In [208]:
# Concaténer les valeurs de "CTS Code" et "CTS Name" avec un séparateur "-"
tax["CTS Name"] = tax["CTS Code"] + " - " + tax["CTS Name"]

In [209]:
# Supprimer la colonne "CTS Code" puisque ses valeurs ont été concaténées avec "CTS Name"
# Supprimer les colonnes "ObjectId", "ISO2", "Indicator", "Source", "CTS Full Descriptor"
tax.drop(columns=["CTS Code", "ObjectId", "ISO2", "Indicator", "Source", "CTS Full Descriptor"], inplace=True)

In [210]:
tax.head(5)

,Country,ISO3,CTS Name,Unit,Année,Taxe
0,Albania,ALB,ECGTE - Environmental Taxes,Domestic Currency,1995,NaN
1,Albania,ALB,ECGTE - Environmental Taxes,Percent of GDP,1995,NaN
2,Albania,ALB,ECGTEN - Taxes on Energy (Including Fuel for T...,Domestic Currency,1995,NaN
3,Albania,ALB,ECGTEN - Taxes on Energy (Including Fuel for T...,Percent of GDP,1995,NaN
4,Albania,ALB,ECGTEP - Taxes on Pollution,Domestic Currency,1995,NaN


In [211]:
# Mettre les valeurs de la colonne "Unit" en colonnes distinctes
#tax = tax.pivot_table(index=["Country", "ISO3", "CTS Name", "Année"], columns="Unit", values="Taxe").reset_index()
#tax.head(5)

In [212]:
# Mettre les valeurs de la colonne "CTS Name" en colonnes distinctes
tax = tax.pivot_table(index=["Country", "ISO3", "Année", "Unit"], columns="CTS Name", values="Taxe").reset_index()

In [213]:
tax.head(5)

CTS Name,Country,ISO3,Année,Unit,ECGTE - Environmental Taxes,ECGTEN - Taxes on Energy (Including Fuel for Transport),ECGTEP - Taxes on Pollution,ECGTER - Taxes on Resources,ECGTET - Taxes on Transport (Excluding Fuel for Transport)
0,Albania,ALB,2015,Domestic Currency,4.399314e+10,3.774111e+10,1.782069e+09,3.254649e+07,4.437413e+09
1,Albania,ALB,2015,Percent of GDP,3.067206e+00,2.631314e+00,1.242460e-01,2.269145e-03,3.093769e-01
2,Albania,ALB,2016,Domestic Currency,4.781379e+10,4.094562e+10,1.879970e+09,5.252434e+07,4.935684e+09
3,Albania,ALB,2016,Percent of GDP,3.247163e+00,2.780726e+00,1.276738e-01,3.567068e-03,3.351955e-01
4,Albania,ALB,2017,Domestic Currency,4.754858e+10,4.040004e+10,1.941324e+09,6.186136e+07,5.145347e+09


In [214]:
# Pivoter les données pour séparer les valeurs par type d'unité
tax = tax.pivot_table(index=['Country', 'ISO3', 'Année'], columns='Unit')

tax.head(5)

CTS Name           ECGTE - Environmental Taxes                 \
Unit                         Domestic Currency Percent of GDP   
Country ISO3 Année                                              
Albania ALB  2015                 4.399314e+10       3.067206   
             2016                 4.781379e+10       3.247163   
             2017                 4.754858e+10       3.066373   
             2018                 5.114559e+10       3.124865   
             2019                 5.341565e+10       3.157133   

CTS Name           ECGTEN - Taxes on Energy (Including Fuel for Transport)  \
Unit                                                     Domestic Currency   
Country ISO3 Année                                                           
Albania ALB  2015                                        3.774111e+10        
             2016                                        4.094562e+10        
             2017                                        4.040004e+10        
             2018                                        4.352182e+10        
             2019                                        4.516530e+10        

CTS Name                          ECGTEP - Taxes on Pollution                 \
Unit               Percent of GDP           Domestic Currency Percent of GDP   
Country ISO3 Année                                                             
Albania ALB  2015        2.631314                1.782069e+09       0.124246   
             2016        2.780726                1.879970e+09       0.127674   
             2017        2.605369                1.941324e+09       0.125195   
             2018        2.659072                2.226251e+09       0.136018   
             2019        2.669496                2.625011e+09       0.155151   

CTS Name           ECGTER - Taxes on Resources                 \
Unit                         Domestic Currency Percent of GDP   
Country ISO3 Année                                              
Albania ALB  2015                  32546493.00       0.002269   
             2016                  52524339.00       0.003567   
             2017                  61861356.38       0.003989   
             2018                  56011991.00       0.003422   
             2019                  58777099.50       0.003474   

CTS Name           ECGTET - Taxes on Transport (Excluding Fuel for Transport)  \
Unit                                                        Domestic Currency   
Country ISO3 Année                                                              
Albania ALB  2015                                        4.437413e+09           
             2016                                        4.935684e+09           
             2017                                        5.145347e+09           
             2018                                        5.341506e+09           
             2019                                        5.566564e+09           

CTS Name                           
Unit               Percent of GDP  
Country ISO3 Année                 
Albania ALB  2015        0.309377  
             2016        0.335195  
             2017        0.331820  
             2018        0.326352  
             2019        0.329012

In [215]:
# Définir une fonction de formatage en fonction du type d'unité
def format_value(value, unit):
    if "Domestic Currency" in unit:
        return '{:,.2f}'.format(value).replace(',', ' ')
    elif "Percent of GDP" in unit:
        return '{:.4f}%'.format(value)
    else:
        return value  # Garantir le retour de la valeur d'origine si l'unité n'est pas reconnue

# Appliquer le formatage en fonction du type d'unité pour chaque colonne concernée
for column in tax.columns:
    if "Domestic Currency" in column:
        tax[column] = tax[column].apply(lambda x: format_value(x, column))
    elif "Percent of GDP" in column:
        tax[column] = tax[column].apply(lambda x: format_value(x, column))

tax.head(5)

CTS Name           ECGTE - Environmental Taxes                 \
Unit                         Domestic Currency Percent of GDP   
Country ISO3 Année                                              
Albania ALB  2015            43 993 138 348.27        3.0672%   
             2016            47 813 792 742.97        3.2472%   
             2017            47 548 576 614.95        3.0664%   
             2018            51 145 591 824.17        3.1249%   
             2019            53 415 648 050.56        3.1571%   

CTS Name           ECGTEN - Taxes on Energy (Including Fuel for Transport)  \
Unit                                                     Domestic Currency   
Country ISO3 Année                                                           
Albania ALB  2015                                   37 741 109 827.00        
             2016                                   40 945 615 341.00        
             2017                                   40 400 044 445.82        
             2018                                   43 521 821 908.60        
             2019                                   45 165 295 551.00        

CTS Name                          ECGTEP - Taxes on Pollution                 \
Unit               Percent of GDP           Domestic Currency Percent of GDP   
Country ISO3 Année                                                             
Albania ALB  2015         2.6313%            1 782 068 792.97        0.1242%   
             2016         2.7807%            1 879 969 501.87        0.1277%   
             2017         2.6054%            1 941 324 188.75        0.1252%   
             2018         2.6591%            2 226 251 450.57        0.1360%   
             2019         2.6695%            2 625 011 287.71        0.1552%   

CTS Name           ECGTER - Taxes on Resources                 \
Unit                         Domestic Currency Percent of GDP   
Country ISO3 Année                                              
Albania ALB  2015                32 546 493.00        0.0023%   
             2016                52 524 339.00        0.0036%   
             2017                61 861 356.38        0.0040%   
             2018                56 011 991.00        0.0034%   
             2019                58 777 099.50        0.0035%   

CTS Name           ECGTET - Taxes on Transport (Excluding Fuel for Transport)  \
Unit                                                        Domestic Currency   
Country ISO3 Année                                                              
Albania ALB  2015                                    4 437 413 235.30           
             2016                                    4 935 683 561.10           
             2017                                    5 145 346 624.00           
             2018                                    5 341 506 474.00           
             2019                                    5 566 564 112.35           

CTS Name                           
Unit               Percent of GDP  
Country ISO3 Année                 
Albania ALB  2015         0.3094%  
             2016         0.3352%  
             2017         0.3318%  
             2018         0.3264%  
             2019         0.3290%

In [216]:
tax.to_csv('./output/tax.csv', index=False)

In [125]:
# Définir une fonction pour formater les valeurs en fonction de la colonne "Unit"
def formatter(value, unit):
    if unit == "Domestic Currency":
        # Formater la valeur avec séparateur d'espace et arrondi à 2 chiffres après la virgule
        return '{:,.2f}'.format(value).replace(',', ' ')
    elif unit == "Percent of GDP":
        # Formater la valeur en pourcentage avec 4 chiffres après la virgule
        return '{:.4f}%'.format(value)
    else:
        return value

# Appliquer la fonction à chaque cellule du dataframe en fonction de la colonne "Unit"
for col in tax.select_dtypes(include=['float64']).columns:
    tax[col] = tax.apply(lambda row: formatter(row[col], row['Unit']), axis=1)



In [58]:
tax.head(5)

CTS Name,Country,ISO3,Année,Unit,ECGTE - Environmental Taxes,ECGTEN - Taxes on Energy (Including Fuel for Transport),ECGTEP - Taxes on Pollution,ECGTER - Taxes on Resources,ECGTET - Taxes on Transport (Excluding Fuel for Transport)
0,Albania,ALB,2015,Domestic Currency,43 993 138 348.27,37 741 109 827.00,1 782 068 792.97,32 546 493.00,4 437 413 235.30
1,Albania,ALB,2015,Percent of GDP,3.0672%,2.6313%,0.1242%,0.0023%,0.3094%
2,Albania,ALB,2016,Domestic Currency,47 813 792 742.97,40 945 615 341.00,1 879 969 501.87,52 524 339.00,4 935 683 561.10
3,Albania,ALB,2016,Percent of GDP,3.2472%,2.7807%,0.1277%,0.0036%,0.3352%
4,Albania,ALB,2017,Domestic Currency,47 548 576 614.95,40 400 044 445.82,1 941 324 188.75,61 861 356.38,5 145 346 624.00


In [59]:
tax.shape

(5494, 9)

In [62]:
# Pivoter les données pour séparer les valeurs par type d'unité
tax = tax.pivot_table(index=['Country', 'ISO3', 'Année'], columns='Unit', values=['ECGTE - Environmental Taxes', 'ECGTEN - Taxes on Energy (Including Fuel for Transport)', 'ECGTEP - Taxes on Pollution', 'ECGTER - Taxes on Resources', 'ECGTET - Taxes on Transport (Excluding Fuel for Transport)'])

# Réinitialiser l'index pour avoir un DataFrame plat
tax.reset_index(inplace=True)

# Renommer les colonnes pour correspondre au format souhaité
tax.columns = ['Country', 'ISO3', 'Année', 'ECGTE - Environmental Taxes by Domestic Currency', 'ECGTE - Environmental Taxes by Percent of GDP', 'ECGTEN - Taxes on Energy (Including Fuel for Transport) by Domestic Currency', 'ECGTEN - Taxes on Energy (Including Fuel for Transport) by Percent of GDP', 'ECGTEP - Taxes on Pollution by Domestic Currency', 'ECGTEP - Taxes on Pollution by Percent of GDP', 'ECGTER - Taxes on Resources by Domestic Currency', 'ECGTER - Taxes on Resources by Percent of GDP', 'ECGTET - Taxes on Transport (Excluding Fuel for Transport) by Domestic Currency', 'ECGTET - Taxes on Transport (Excluding Fuel for Transport) by Percent of GDP']

TypeError: agg function failed [how->mean,dtype->object]

In [45]:
tax.to_csv('./output/Tax_Energy_clear.csv', index=False)